## Tools to test and calibrate

In [ ]:
#!/usr/bin/python
import minimalmodbus
import time
import socket


#Setup UDP server
SERVER_IP    = ""
SERVER_PORT  = 8000
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((SERVER_IP, SERVER_PORT))
#print(f"Listening for clients")

#Setup RS485
port         = '/dev/ttySC0'
address      = 1
debug        = False
baudrate     = 9600
instrument = minimalmodbus.Instrument(port, address, debug = debug)  # port name, slave add>
instrument.serial.baudrate = baudrate                                # Baud
instrument.serial.bytesize = 8                                       # bytesize
instrument.serial.stopbits = 1                                       # stop bits
instrument.serial.timeout  = 0.5                                     # seconds
instrument.mode = minimalmodbus.MODE_RTU                             # rtu or ascii mode
instrument.clear_buffers_beforetouch_each_transaction = True


def twos_compliment_16(value):
    return -(value & 0x8000) | (value & 0x7fff)

while True:
    data, address = sock.recvfrom(4096)
    if data=='get data'.encode():
       # print(f"Received {data} from {address}")
        sign=1
        running_state = int(instrument.read_register(13001-1, 0, functioncode=4))
        battery_current = instrument.read_register(13021-1, 1, functioncode=4)
        battery_SoC = instrument.read_register(13023-1, 1, functioncode=4)
        load_power = twos_compliment_16(instrument.read_register(13008-1,0,functioncode=4))
        if (running_state & 0x04):
            sign=-1
        message = str(sign*battery_current)  +','+str(battery_SoC) +','+str(load_power)
        sock.sendto(message.encode(), (address))

    if data=='battery current'.encode():
       # print(f"Received {data} from {address}")
        sign=1
        running_state = int(instrument.read_register(13001-1, 0, functioncode=4))
        battery_current = instrument.read_register(13021-1, 1, functioncode=4)
        #battery_SoC = instrument.read_register(13023-1, 1, functioncode=4)
        #load_power = twos_compliment_16(instrument.read_register(13008-1,0,functioncode=4)>
        if (running_state & 0x04):
            sign=-1
        message = str(sign*battery_current) #  +','+str(battery_SoC) +','+str(load_power)
        sock.sendto(message.encode(), (address))

    if data=='test 10'.encode():
       # print(f"Received {data} from {address}")
        sign=1
        running_state = int(instrument.read_register(13001-1, 0, functioncode=4))
        battery_current = instrument.read_register(13021-1, 1, functioncode=4)
        #battery_SoC = instrument.read_register(13023-1, 1, functioncode=4)
        #load_power = twos_compliment_16(instrument.read_register(13008-1,0,functioncode=4)>
        if (running_state & 0x04):
            sign=-1
        message =str(10.0)
        sock.sendto(message.encode(), (address))

    if data=='test 50'.encode():
       # print(f"Received {data} from {address}")
        sign=1
        running_state = int(instrument.read_register(13001-1, 0, functioncode=4))
        battery_current = instrument.read_register(13021-1, 1, functioncode=4)
        #battery_SoC = instrument.read_register(13023-1, 1, functioncode=4)
        #load_power = twos_compliment_16(instrument.read_register(13008-1,0,functioncode=4)>
        if (running_state & 0x04):
            sign=-1
        message =str(50.0) 
        sock.sendto(message.encode(), (address))  


    if data=='test -50'.encode():
       # print(f"Received {data} from {address}")
        sign=1
        running_state = int(instrument.read_register(13001-1, 0, functioncode=4))
        battery_current = instrument.read_register(13021-1, 1, functioncode=4)
        #battery_SoC = instrument.read_register(13023-1, 1, functioncode=4)
        #load_power = twos_compliment_16(instrument.read_register(13008-1,0,functioncode=4)>
        if (running_state & 0x04):
            sign=-1
        message =str(-50.0) 
        sock.sendto(message.encode(), (address))  


### Extract and display data from Raspberry Pi server

In [ ]:
import time
import socket
import datetime


SERVER_IP    = "10.0.1.69"   # insert your own Raspberry Pi address
SERVER_PORT  = 8000         

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

while True:
    start_time = datetime.datetime.now()
    message = 'get data'
    sock.sendto(message.encode(), (SERVER_IP, SERVER_PORT))
    data, address = sock.recvfrom(4096)
    end_time = datetime.datetime.now()
    
    if data:
        now = datetime.datetime.now()
        data_string = data.decode().split(',')
        battery_current = float(data_string[0])
        battery_voltage = float(data_string[1])
        battery_SoC = float(data_string[2])
        load_power = float(data_string[3])
        battery_power = float(data_string[4])         ####  <- this is the key number for the Zappi
        print('                                                                                      ',end='\r')
        print(now.strftime("%H:%M:%S"),'{:6.1f}A   {:6.1f}V    SoC {:5.1f}%    Load {:4,.0f}W    Charge {:4,.0f}W'
              .format(battery_current,battery_voltage,battery_SoC,load_power,battery_power),end='\r')

    time.sleep(5)



### Set fixed response for calibration

In [22]:
# Recommend plotting set value vs value observerd on myenergi app.  
# Only decimal place available so line of best fit is a good choice for factor


import time
import socket
import datetime


SERVER_IP    = "10.0.1.69"
SERVER_PORT  = 8000

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
message = 'clear'       # remember to 'clear' when finsihed or the battery will indicate this number forever!
#message = 'set 1000'   number is watts
#message = 'set 2500'
sent = sock.sendto(message.encode(), (SERVER_IP, SERVER_PORT))
print(sent, 'bytes transmitted')



5 bytes transmitted


In [ ]:
import numpy as np
import matplotlib.pyplot as plt



amplitude = 12
offset = 15
frequency = 50


end_time = 500       # msec
sample_time = 500    # micro secs
number_of_points = int(end_time*1000/sample_time)
time_points = np.linspace(0,end_time,number_of_points)

AC_ref = amplitude * np.sin(2*np.pi*frequency*time_points) + offset
fig1 = plt.figure(figsize=(8,6))
ax = fig1.add_subplot(111)
ax.plot(time_points,AC_ref)
plt.show()

total = 0.
total_squared = 0.

for point in AC_ref:
    total = total + point
    total_squared = total_squared + point*point

RMS = np.sqrt(total_squared/number_of_points)
mean = total/number_of_points

print('RMS {:4.1f}   mean {:4.1f}'.format(RMS,mean))
